In [9]:
import pandas as pd

df1 = pd.DataFrame({'HPI':[80,85,88,85],
                    'Int_rate':[2, 3, 2, 2],
                    'US_GDP_Thousands':[50, 55, 65, 55]},
                   index = [2001, 2002, 2003, 2004])

df2 = pd.DataFrame({'HPI':[80,85,88,85],
                    'Int_rate':[2, 3, 2, 2],
                    'US_GDP_Thousands':[50, 55, 65, 55]},
                   index = [2005, 2006, 2007, 2008])

df3 = pd.DataFrame({'HPI':[80,85,88,85],
                    'Unemployment':[7, 8, 9, 6],
                    'Low_tier_HPI':[50, 52, 50, 53]},
                   index = [2001, 2002, 2003, 2004])

In [10]:
#Here we are sharing column 'HPI'
print(pd.merge(df1,df3, on='HPI'))

   HPI  Int_rate  US_GDP_Thousands  Low_tier_HPI  Unemployment
0   80         2                50            50             7
1   85         3                55            52             8
2   85         3                55            53             6
3   85         2                55            52             8
4   85         2                55            53             6
5   88         2                65            50             9


In [11]:
#We can share multiple columns also
print(pd.merge(df1,df2, on=['HPI','Int_rate']))

   HPI  Int_rate  US_GDP_Thousands_x  US_GDP_Thousands_y
0   80         2                  50                  50
1   85         3                  55                  55
2   88         2                  65                  65
3   85         2                  55                  55


### Here there are two versions of the US_GDP_Thousands. This is because we didn't share on these columns, so both are retained, with another letter to differentiate.

In [12]:
df4 = pd.merge(df1,df3, on='HPI')
df4.set_index('HPI', inplace=True)
print(df4)

     Int_rate  US_GDP_Thousands  Low_tier_HPI  Unemployment
HPI                                                        
80          2                50            50             7
85          3                55            52             8
85          3                55            53             6
85          2                55            52             8
85          2                55            53             6
88          2                65            50             9


### What if 'HPI' was already an index ? In this case we would probably use 'join()'

In [13]:
df1.set_index('HPI', inplace=True)        #Making 'HPI' as index
df3.set_index('HPI', inplace=True)        #Making 'HPI' as index
joined = df1.join(df3)
print(joined)

     Int_rate  US_GDP_Thousands  Low_tier_HPI  Unemployment
HPI                                                        
80          2                50            50             7
85          3                55            52             8
85          3                55            53             6
85          2                55            52             8
85          2                55            53             6
88          2                65            50             9


In [14]:
# let's consider joining and merging on slightly differing indexes.
# We will redefine dataframes df1 and df3 from starting

df1 = pd.DataFrame({
                    'Int_rate':[2, 3, 2, 2],
                    'US_GDP_Thousands':[50, 55, 65, 55],
                    'Year':[2001, 2002, 2003, 2004]
                    })

df3 = pd.DataFrame({
                    'Unemployment':[7, 8, 9, 6],
                    'Low_tier_HPI':[50, 52, 50, 53],
                    'Year':[2001, 2003, 2004, 2005]})

### Now we have similar year columns, but different dates. df3 has 2005 but not 2002, and df1 is the reverse of that. What happens when we merge?

In [15]:
merged = pd.merge(df1,df3, on='Year')
print(merged)

   Int_rate  US_GDP_Thousands  Year  Low_tier_HPI  Unemployment
0         2                50  2001            50             7
1         2                65  2003            52             8
2         2                55  2004            50             9


In [16]:
#This one is a bit more useful I think by setting the index as 'Year'

merged.set_index('Year', inplace=True)
print(merged)

      Int_rate  US_GDP_Thousands  Low_tier_HPI  Unemployment
Year                                                        
2001         2                50            50             7
2003         2                65            52             8
2004         2                55            50             9


### Notice how 2005 and 2002 are just totally missing? Merge will natively just merge existing/shared data. What can we do about this? It turns out, there is a "how" parameter when merging. This parameter reflects the merging choices that come from merging databases.

#### We have the following choices: Left, right, outer inner.

##### 1) Left - equal to left outer join SQL - use keys from left frame only
##### 2) Right - right outer join from SQL- use keys from right frame only.
##### 3) Outer - full outer join - use union of keys
##### 4) Inner - use only intersection of keys.

In [17]:
merged = pd.merge(df1,df3, on='Year', how='left')
merged.set_index('Year', inplace=True)
print(merged)

# Left dataframe is df1 and it had the year 2001,2002,2003,2004. So we got an index 
# that is identical to left dataframe df1. 
# In merge() left dataframe is df1 and right dataframe is df3

      Int_rate  US_GDP_Thousands  Low_tier_HPI  Unemployment
Year                                                        
2001         2                50          50.0           7.0
2002         3                55           NaN           NaN
2003         2                65          52.0           8.0
2004         2                55          50.0           9.0


In [18]:
merged = pd.merge(df1,df3, on='Year', how='right')
merged.set_index('Year', inplace=True)
print(merged)

# Here we used right dataframe i.e. df3 which can be seen in merge() above.

      Int_rate  US_GDP_Thousands  Low_tier_HPI  Unemployment
Year                                                        
2001       2.0              50.0            50             7
2003       2.0              65.0            52             8
2004       2.0              55.0            50             9
2005       NaN               NaN            53             6


In [19]:
merged = pd.merge(df1,df3, on='Year', how='outer')
merged.set_index('Year', inplace=True)
print(merged)

# Here we used 'outer' which is a union of the keys. This means all of the indexes are shown.

      Int_rate  US_GDP_Thousands  Low_tier_HPI  Unemployment
Year                                                        
2001       2.0              50.0          50.0           7.0
2002       3.0              55.0           NaN           NaN
2003       2.0              65.0          52.0           8.0
2004       2.0              55.0          50.0           9.0
2005       NaN               NaN          53.0           6.0


In [20]:
merged = pd.merge(df1,df3, on='Year', how='inner')
merged.set_index('Year', inplace=True)
print(merged)

# Here we used 'inner' which is intersection of keys, basically what is shared between all the sets

#From this we can conclude that the default option is 'inner'

      Int_rate  US_GDP_Thousands  Low_tier_HPI  Unemployment
Year                                                        
2001         2                50            50             7
2003         2                65            52             8
2004         2                55            50             9


In [21]:
#Now we can check out joining, which will join on the index

df1.set_index('Year', inplace=True)
df3.set_index('Year', inplace=True)
joined = df1.join(df3, how="outer")
print(joined)

      Int_rate  US_GDP_Thousands  Low_tier_HPI  Unemployment
Year                                                        
2001       2.0              50.0          50.0           7.0
2002       3.0              55.0           NaN           NaN
2003       2.0              65.0          52.0           8.0
2004       2.0              55.0          50.0           9.0
2005       NaN               NaN          53.0           6.0
